In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
teamexp = pd.read_csv("/Users/olivianewton/Documents/Manuscripts/TopiCS_SpecialIssue/team_expertise_info_raw.csv")

In [ ]:
ax = sns.distplot(teamexp[teamexp['Team type'] == 'human']['followers'])
ax = sns.distplot(teamexp[teamexp['Team type'] == 'human-bot']['followers'])

In [ ]:
teamexp[teamexp['Team type'] == 'human'].mean()

In [ ]:
teamexp[teamexp['Team type'] == 'human-bot'].mean()

In [ ]:
from sklearn.preprocessing import StandardScaler
# print(teamexp.columns)
scaler = StandardScaler()
cols_to_normalize = ['followers', 'following', 'public_repos', 'gh_impact']
X = scaler.fit_transform(teamexp[cols_to_normalize])
normalized_teams = pd.DataFrame(data=X, index=teamexp['name_h'], columns=cols_to_normalize)
normalized_teams.reset_index(inplace=True)
temp = teamexp[['name_h', 'Team type']]
# print(temp.columns, normalized_teams.columns)
normalized_teams = normalized_teams.merge(temp, on='name_h')
normalized_teams.head()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
# normalized_teams.set_index('name_h', inplace=True)
X = normalized_teams
coltore = ['Team type', 'name_h']
print(X.columns)
X_hb = X[X['Team type'] == 'human-bot']
X_h = X[X['Team type'] == 'human']
# nbrs = NearestNeighbors(n_neighbors=1, algorithm='brute').fit(X_h.drop(coltore, axis=1))
human_teams = X_h.set_index('name_h').drop('Team type', axis=1).copy(deep=True)
selected_teams = []
for i, row in X_hb.drop(coltore, axis=1).iterrows():
    distances = euclidean_distances(row.values.reshape(1, -1), human_teams)
    min_idx = np.argmin(distances)
    closest_team = human_teams.index.tolist()[min_idx]
    selected_teams.append(closest_team)
    human_teams.drop(closest_team, inplace=True)
#     print(human_teams.shape)

In [ ]:
X_h.reset_index(inplace=True)
X_h_downsampled = X_h[X_h['name_h'].isin(selected_teams)]
print(X_h_downsampled.shape, X_h_downsampled.columns)

In [ ]:
X_h_downsampled.mean()

In [ ]:
X_hb.mean()

In [ ]:
X_h_downsampled['name_h'].to_csv("/Users/olivianewton/Documents/Manuscripts/TopiCS_SpecialIssue/Human_Teams_SmartSampled.csv", index=False)